<a href="https://colab.research.google.com/github/mongvuot336699/BT-matrix/blob/master/trung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import warnings
warnings.filterwarnings('ignore')
from scipy.sparse import hstack, csr_matrix

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
# data_train = pd.read_csv('train.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head(2)
test.head(2)


,No,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,...,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,33858,No,No,United States,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",NaN,Back-end developer;Front-end developer;Full-st...,9-11 years,3-5 years,Neither satisfied nor dissatisfied,Slightly satisfied,Doing the same work,"I’m not actively looking, but I am open to new...",More than 4 years ago,9.0,8.0,6.0,3.0,1.0,5.0,4.0,2.0,10.0,7.0,1.0,7.0,3.0,10.0,5.0,6.0,8.0,2.0,11.0,9.0,4.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33859,Yes,No,United States,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",NaN,100 to 499 employees,Back-end developer;Database administrator;DevO...,3-5 years,3-5 years,Extremely satisfied,Extremely satisfied,Doing the same work,"I’m not actively looking, but I am open to new...",Less than a year ago,10.0,4.0,9.0,1.0,2.0,8.0,7.0,3.0,6.0,5.0,1.0,10.0,2.0,8.0,11.0,4.0,7.0,6.0,5.0,9.0,3.0,2.0,1.0,...,5.0,7.0,4.0,2.0,Artificial intelligence surpassing human intel...,Increasing automation of jobs,The developers or the people creating the AI,I'm excited about the possibilities more than ...,Depends on what it is,Depends on what it is,Upper management at the company/organization,Yes,10 (Very Likely),Multiple times per day,Yes,Multiple times per day,Yes,"No, I have one but it's out of date",10 (Very Likely),Yes,Somewhat interested,Very interested,Very interested,Extremely interested,Extremely interested,Between 5:00 - 6:00 AM,9 - 12 hours,1 - 2 hours,Never,NaN,Daily or almost every day,Male,Straight or heterosexual,"Master’s degree (MA, MS, M.Eng., MBA, etc.)",White or of European descent,25 - 34 years old,Yes,No,The survey was an appropriate length,Very easy


In [6]:
train.shape

(33857, 128)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33857 entries, 0 to 33856
Columns: 128 entries, No to SurveyEasy
dtypes: float64(41), int64(1), object(86)
memory usage: 33.1+ MB


In [8]:
train.isnull().any().sum()

122

In [9]:
train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33857 entries, 0 to 33856
Data columns (total 128 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   No                           int64  
 1   Hobby                        object 
 2   OpenSource                   object 
 3   Country                      object 
 4   Student                      object 
 5   Employment                   object 
 6   FormalEducation              object 
 7   UndergradMajor               object 
 8   CompanySize                  object 
 9   DevType                      object 
 10  YearsCoding                  object 
 11  YearsCodingProf              object 
 12  JobSatisfaction              object 
 13  CareerSatisfaction           object 
 14  HopeFiveYears                object 
 15  JobSearchStatus              object 
 16  LastNewJob                   object 
 17  AssessJob1                   float64
 18  AssessJob2                   float64
 19  Ass

<h2> Text - preprocessing </h2>


In [10]:
df_train = train[['Country', 'Student', 'Employment', 'FormalEducation', 'UndergradMajor', 'CompanySize','DevType',
                  'YearsCoding','YearsCodingProf',
                  'UpdateCV','Currency','Salary','CurrencySymbol','CommunicationTools',
                  'TimeFullyProductive','EducationTypes','SelfTaughtTypes','HackathonReasons',
                  'LanguageWorkedWith','DatabaseWorkedWith',
                  'PlatformWorkedWith','FrameworkWorkedWith'
                  ,'NumberMonitors','Methodology','AIDangerous'
                  ,'AIResponsible','WakeTime','HoursOutside','Exercise','Gender',
                  'EducationParents','Age','Dependents']]
df_test = test[['Country', 'Student', 'Employment', 'FormalEducation', 'UndergradMajor', 'CompanySize','DevType',
                  'YearsCoding','YearsCodingProf',
                  'UpdateCV','Currency','CurrencySymbol','CommunicationTools',
                  'TimeFullyProductive','EducationTypes','SelfTaughtTypes','HackathonReasons',
                  'LanguageWorkedWith','DatabaseWorkedWith',
                  'PlatformWorkedWith','FrameworkWorkedWith'
                  ,'NumberMonitors','Methodology','AIDangerous'
                  ,'AIResponsible','WakeTime','HoursOutside','Exercise','Gender',
                  'EducationParents','Age','Dependents']]
df_train.head(2)
df_test.head(2)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33857 entries, 0 to 33856
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              33857 non-null  object 
 1   Student              33563 non-null  object 
 2   Employment           33741 non-null  object 
 3   FormalEducation      33370 non-null  object 
 4   UndergradMajor       30154 non-null  object 
 5   CompanySize          29075 non-null  object 
 6   DevType              33624 non-null  object 
 7   YearsCoding          33843 non-null  object 
 8   YearsCodingProf      33341 non-null  object 
 9   UpdateCV             32501 non-null  object 
 10  Currency             30564 non-null  object 
 11  Salary               33857 non-null  float64
 12  CurrencySymbol       33857 non-null  object 
 13  CommunicationTools   30200 non-null  object 
 14  TimeFullyProductive  28310 non-null  object 
 15  EducationTypes       32552 non-null 

In [11]:
def clean_country(country):
  country = str(country).lower()
  country = re.sub(r'\...','', country)
  country = re.sub(r'â€™','', country)
  return country

df_train['FormalEducation'] = df_train['FormalEducation'].apply(clean_country)

df_train['UpdateCV'] = df_train['UpdateCV'].apply(clean_country)
df_train['SelfTaughtTypes'] = df_train['SelfTaughtTypes'].apply(clean_country)
df_train['EducationParents'] = df_train['EducationParents'].apply(clean_country)

df_test['FormalEducation'] = df_test['FormalEducation'].apply(clean_country)

df_test['UpdateCV'] = df_test['UpdateCV'].apply(clean_country)
df_test['SelfTaughtTypes'] = df_test['SelfTaughtTypes'].apply(clean_country)
df_test['EducationParents'] = df_test['EducationParents'].apply(clean_country)

df_test.head(2)
df_train.head(2)


,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,UpdateCV,Currency,Salary,CurrencySymbol,CommunicationTools,TimeFullyProductive,EducationTypes,SelfTaughtTypes,HackathonReasons,LanguageWorkedWith,DatabaseWorkedWith,PlatformWorkedWith,FrameworkWorkedWith,NumberMonitors,Methodology,AIDangerous,AIResponsible,WakeTime,HoursOutside,Exercise,Gender,EducationParents,Age,Dependents
0,Nigeria,No,"Independent contractor, freelancer, or self-em...","bachelor’s degree (ba, bs, bgetc.)","Computer science, computer engineering, or sof...",NaN,Database administrator;Designer;Full-stack dev...,18-20 years,6-8 years,my job status or other personal status changed,U.S. dollars ($),10800.0,USD,Facebook;Office / productivity suite (Microsof...,NaN,Taken a part-time in-person course in programm...,the official documentation and/or standards fo...,NaN,PHP;SQL;HTML;CSS,MySQL,Linux;Windows Desktop or Server;WordPress,NaN,1,Formal standard such as ISO 9001 or IEEE 12207...,Increasing automation of jobs,The developers or the people creating the AI,Between 5:00 - 6:00 AM,Over 4 hours,I don't typically exercise,Male,"bachelor’s degree (ba, bs, bgetc.)",35 - 44 years old,Yes
1,France,No,Employed full-time,"master’s degree (ma, ms, mgmba, etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Back-end developer,18-20 years,18-20 years,a friend told me about a job opportunity,Euros (€),73433.0,EUR,Confluence;Google Hangouts/Chat;HipChat;Jira;S...,More than a year,Taken an online course in programming or softw...,the official documentation and/or standards fo...,NaN,Go;Groovy;Java;JavaScript;Bash/Shell,MySQL;PostgreSQL,Linux,NaN,2,Agile;Extreme programming (XP);Formal standard...,"Evolving definitions of ""fairness"" in algorith...",The developers or the people creating the AI,Between 7:01 - 8:00 AM,30 - 59 minutes,I don't typically exercise,Male,"master’s degree (ma, ms, mgmba, etc.)",35 - 44 years old,Yes


In [12]:
df_train['Currency'].fillna('missing', inplace = True)
df_test['Currency'].fillna('missing', inplace = True)
df_train.head(10)

,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,UpdateCV,Currency,Salary,CurrencySymbol,CommunicationTools,TimeFullyProductive,EducationTypes,SelfTaughtTypes,HackathonReasons,LanguageWorkedWith,DatabaseWorkedWith,PlatformWorkedWith,FrameworkWorkedWith,NumberMonitors,Methodology,AIDangerous,AIResponsible,WakeTime,HoursOutside,Exercise,Gender,EducationParents,Age,Dependents
0,Nigeria,No,"Independent contractor, freelancer, or self-em...","bachelor’s degree (ba, bs, bgetc.)","Computer science, computer engineering, or sof...",NaN,Database administrator;Designer;Full-stack dev...,18-20 years,6-8 years,my job status or other personal status changed,U.S. dollars ($),10800.0,USD,Facebook;Office / productivity suite (Microsof...,NaN,Taken a part-time in-person course in programm...,the official documentation and/or standards fo...,NaN,PHP;SQL;HTML;CSS,MySQL,Linux;Windows Desktop or Server;WordPress,NaN,1,Formal standard such as ISO 9001 or IEEE 12207...,Increasing automation of jobs,The developers or the people creating the AI,Between 5:00 - 6:00 AM,Over 4 hours,I don't typically exercise,Male,"bachelor’s degree (ba, bs, bgetc.)",35 - 44 years old,Yes
1,France,No,Employed full-time,"master’s degree (ma, ms, mgmba, etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Back-end developer,18-20 years,18-20 years,a friend told me about a job opportunity,Euros (€),73433.0,EUR,Confluence;Google Hangouts/Chat;HipChat;Jira;S...,More than a year,Taken an online course in programming or softw...,the official documentation and/or standards fo...,NaN,Go;Groovy;Java;JavaScript;Bash/Shell,MySQL;PostgreSQL,Linux,NaN,2,Agile;Extreme programming (XP);Formal standard...,"Evolving definitions of ""fairness"" in algorith...",The developers or the people creating the AI,Between 7:01 - 8:00 AM,30 - 59 minutes,I don't typically exercise,Male,"master’s degree (ma, ms, mgmba, etc.)",35 - 44 years old,Yes
2,United Kingdom,No,Employed full-time,some college/university study without earning ...,"Computer science, computer engineering, or sof...",NaN,Designer;Full-stack developer,27-29 years,15-17 years,my job status or other personal status changed,British pounds sterling (£),66672.0,GBP,"Office / productivity suite (Microsoft Office,...",One to three months,"Taught yourself a new language, framework, or ...",the official documentation and/or standards fo...,NaN,C++;JavaScript;SQL;HTML;CSS,NaN,Linux,Node.js,2,Agile;Scrum,Algorithms making important decisions,A governmental or other regulatory body,Between 6:01 - 7:00 AM,1 - 2 hours,I don't typically exercise,Male,"secondary school (e american high school, germ...",35 - 44 years old,No
3,Russian Federation,No,Employed full-time,"bachelor’s degree (ba, bs, bgetc.)","Computer science, computer engineering, or sof...",100 to 499 employees,Back-end developer;Desktop or enterprise appli...,9-11 years,9-11 years,i had a negative experience or interaction at ...,Russian rubles (₽),31848.0,RUB,"Other chat system (IRC, proprietary software, ...",One to three months,Taken an online course in programming or softw...,the official documentation and/or standards fo...,To improve my general technical skills or prog...,Assembly;C;C++;Go;Python;Bash/Shell,NaN,Linux,NaN,2,NaN,Algorithms making important decisions,Nobody,Between 9:01 - 10:00 AM,30 - 59 minutes,I don't typically exercise,Male,primary/elementary school,25 - 34 years old,Yes
4,Germany,No,"Independent contractor, freelancer, or self-em...",primary/elementary school,NaN,NaN,Back-end developer;Game or graphics developer,6-8 years,0-2 years,a friend told me about a job opportunity,U.S. dollars ($),0.0,USD,"Other chat system (IRC, proprietary software, ...",NaN,"Taught yourself a new language, framework, or ...",the official documentation and/or standards fo...,NaN,C#;Java;JavaScript;Lua;PHP;SQL;VB.NET;HTML,MySQL,Arduino;Linux;Raspberry Pi;Windows Desktop or ...,.NET Core;Node.js,2,N

In [13]:
def min_exp(exp):
    val = re.sub(r'\-',' ',str(exp))
    val = val.split(' ')
    val = str(val[0])
    return val 

df_train['min_exp'] = df_train['YearsCoding'].apply(min_exp)
df_test['min_exp'] = df_test['YearsCoding'].apply(min_exp)

df_test['min_exp'].replace('', np.nan, inplace=True)
df_train['min_exp'].replace('', np.nan, inplace=True)
print(df_train['min_exp'].head(20))

0     18
1     18
2     27
3      9
4      6
5      6
6      6
7     18
8     12
9      9
10     6
11     3
12    15
13    12
14    18
15     3
16     6
17     0
18     3
19    21
Name: min_exp, dtype: object


In [14]:
def min_exp_pro(exp):
  val = re.sub(r'\-',' ',str(exp))
  val = val.split(' ')
  val = str(val[0])
  return val

df_train['min_exp_pro'] = df_train['YearsCodingProf'].apply(min_exp_pro)
df_test['min_exp_pro'] = df_test['YearsCodingProf'].apply(min_exp_pro)

df_train['min_exp_pro'].replace('', np.nan, inplace=True)

df_train['min_exp_pro'].head(20)

0       6
1      18
2      15
3       9
4       0
5       6
6       6
7       9
8       3
9       3
10      6
11      0
12     15
13     12
14     15
15      0
16      3
17    nan
18      3
19     18
Name: min_exp_pro, dtype: object

In [15]:
df_train.dropna(subset=['min_exp_pro'],inplace= True)
print(df_train['min_exp_pro'])

0         6
1        18
2        15
3         9
4         0
         ..
33852     3
33853     0
33854    12
33855     3
33856    12
Name: min_exp_pro, Length: 33857, dtype: object


In [16]:
df_train = df_train[~df_train['CompanySize'].isin(['Fewer than 10 employees'])]
df_train['CompanySize'].head(10)

0                          NaN
1     10,000 or more employees
2                          NaN
3         100 to 499 employees
4                          NaN
5                          NaN
6           20 to 99 employees
7     1,000 to 4,999 employees
9           20 to 99 employees
10    10,000 or more employees
Name: CompanySize, dtype: object

In [17]:
df_train.dropna(subset = ['CompanySize'],inplace = True)
df_train['CompanySize'].head(10)

1     10,000 or more employees
3         100 to 499 employees
6           20 to 99 employees
7     1,000 to 4,999 employees
9           20 to 99 employees
10    10,000 or more employees
11        100 to 499 employees
13          10 to 19 employees
14        500 to 999 employees
15          20 to 99 employees
Name: CompanySize, dtype: object

In [18]:
def company(exp):
    val = re.sub(r'\to',' ',str(exp))
    val = val.split(' ')
    val = str(val[0])
    return val


df_train['company'] = df_train['CompanySize'].apply(company)
df_test['company'] = df_test['CompanySize'].apply(company)

print(df_train['company'])

1        10,000
3           100
6            20
7         1,000
9            20
          ...  
33850     5,000
33851        10
33852     1,000
33853     1,000
33856    10,000
Name: company, Length: 26262, dtype: object


In [19]:
df_train['Student'].replace('', np.nan, inplace=True)
df_train.dropna(subset=['Student'], inplace=True)
print(df_train['Student'])

1                    No
3                    No
6                    No
7                    No
9                    No
              ...      
33850    Yes, part-time
33851    Yes, full-time
33852    Yes, part-time
33853    Yes, full-time
33856    Yes, part-time
Name: Student, Length: 26076, dtype: object


In [20]:
df_train['UndergradMajor'].replace('', np.nan, inplace=True)
df_train.dropna(subset=['UndergradMajor'], inplace=True)
print(df_train['UndergradMajor'])

1        A natural science (ex. biology, chemistry, phy...
3        Computer science, computer engineering, or sof...
6        Computer science, computer engineering, or sof...
7        Computer science, computer engineering, or sof...
9        Computer science, computer engineering, or sof...
                               ...                        
33850    Another engineering discipline (ex. civil, ele...
33851    Computer science, computer engineering, or sof...
33852    A natural science (ex. biology, chemistry, phy...
33853    Another engineering discipline (ex. civil, ele...
33856    Another engineering discipline (ex. civil, ele...
Name: UndergradMajor, Length: 23824, dtype: object


In [21]:
df_train['DevType'].replace('', np.nan, inplace=True)
df_train.dropna(subset=['DevType'], inplace=True)
print(df_train['DevType'])

1                                       Back-end developer
3        Back-end developer;Desktop or enterprise appli...
6                                       Back-end developer
7        Back-end developer;Front-end developer;Full-st...
9        Back-end developer;Designer;Front-end develope...
                               ...                        
33850                                 Full-stack developer
33851                                 Full-stack developer
33852    Educator or academic researcher;Student;System...
33853    Back-end developer;DevOps specialist;Student;S...
33856    Back-end developer;Database administrator;Engi...
Name: DevType, Length: 23654, dtype: object


In [22]:
df_train['YearsCoding'].replace('', np.nan, inplace=True)
df_train.dropna(subset=['YearsCoding'], inplace=True)
print(df_train['YearsCoding'])

1        18-20 years
3         9-11 years
6          6-8 years
7        18-20 years
9         9-11 years
            ...     
33850      6-8 years
33851     9-11 years
33852      3-5 years
33853      6-8 years
33856    15-17 years
Name: YearsCoding, Length: 23649, dtype: object


In [49]:
null_columns=df_train.columns[df_train.isnull().any()]
df_train[null_columns].isnull().sum()

CommunicationTools      1310
TimeFullyProductive      828
EducationTypes           861
HackathonReasons       14250
LanguageWorkedWith       559
DatabaseWorkedWith      3420
PlatformWorkedWith      3669
FrameworkWorkedWith     7307
NumberMonitors           614
Methodology             3456
AIDangerous             3416
AIResponsible           2403
WakeTime                 782
HoursOutside             820
Exercise                 785
Gender                  1483
Age                     1307
Dependents              1679
dtype: int64

In [27]:

df_train['Salary'].fillna(value=df_train['Salary'].mean(), inplace = True)
df_train['Salary'].head(20)


1     3375
3     1766
6     3577
7     2377
9     2101
10    4425
11     376
13    2197
14    4598
15    1994
16    2101
21    3152
22     822
25    1378
26    1589
27     429
28    1686
30    4136
32    3425
33    1587
Name: Salary, dtype: int64

In [39]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_train['Salary'] = le.fit_transform(df_train['Salary'])
print(df_train['Salary'])



1        3375
3        1766
6        3577
7        2377
9        2101
         ... 
33850    3425
33851    1709
33852     727
33853    1184
33856    3874
Name: Salary, Length: 23649, dtype: int64


In [48]:
df_train.head(3)

,Country,Student,Employment,FormalEducation,UndergradMajor,DevType,UpdateCV,Currency,Salary,CurrencySymbol,CommunicationTools,TimeFullyProductive,EducationTypes,SelfTaughtTypes,HackathonReasons,LanguageWorkedWith,DatabaseWorkedWith,PlatformWorkedWith,FrameworkWorkedWith,NumberMonitors,Methodology,AIDangerous,AIResponsible,WakeTime,HoursOutside,Exercise,Gender,EducationParents,Age,Dependents,min_exp,min_exp_pro,company
1,43,No,Employed full-time,"master’s degree (ma, ms, mgmba, etc.)","A natural science (ex. biology, chemistry, phy...",Back-end developer,a friend told me about a job opportunity,Euros (€),3375,EUR,Confluence;Google Hangouts/Chat;HipChat;Jira;S...,More than a year,Taken an online course in programming or softw...,the official documentation and/or standards fo...,NaN,Go;Groovy;Java;JavaScript;Bash/Shell,MySQL;PostgreSQL,Linux,NaN,2,Agile;Extreme programming (XP);Formal standard...,"Evolving definitions of ""fairness"" in algorith...",The developers or the people creating the AI,Between 7:01 - 8:00 AM,30 - 59 minutes,I don't typically exercise,Male,"master’s degree (ma, ms, mgmba, etc.)",35 - 44 years old,Yes,18,18,"10,000"
3,106,No,Employed full-time,"bachelor’s degree (ba, bs, bgetc.)","Computer science, computer engineering, or sof...",Back-end developer;Desktop or enterprise appli...,i had a negative experience or interaction at ...,Russian rubles (₽),1766,RUB,"Other chat system (IRC, proprietary software, ...",One to three months,Taken an online course in programming or softw...,the official documentation and/or standards fo...,To improve my general technical skills or prog...,Assembly;C;C++;Go;Python;Bash/Shell,NaN,Linux,NaN,2,NaN,Algorithms making important decisions,Nobody,Between 9:01 - 10:00 AM,30 - 59 minutes,I don't typically exercise,Male,primary/elementary school,25 - 34 years old,Yes,9,9,100
6,138,No,Employed full-time,some college/university study without earning ...,"Computer science, computer engineering, or sof...",Back-end developer,my job status or other personal status changed,U.S. dollars ($),3577,USD,Confluence;Jira;Office / productivity suite (M...,Less than a month,"Taught yourself a new language, framework, or ...",the official documentation and/or standards fo...,NaN,Java;JavaScript;Rust;Kotlin;HTML;CSS,PostgreSQL,AWS,Spring,2,Agile;Kanban;Scrum,Artificial intelligence surpassing human intel...,The developers or the people creating the AI,Between 6:01 - 7:00 AM,Less than 30 minutes,I don't typically exercise,Male,"bachelor’s degree (ba, bs, bgetc.)",25 - 34 years old,No,6,6,20


In [45]:
# df_train = df_train.drop(columns=['CompanySize','YearsCoding','YearsCodingProf'])

KeyError: ignored